In [56]:
import glob
import os
import os.path as osp
import shutil
import xml.etree.ElementTree as ET
import pprint
import json

In [57]:
# 새롭게 만들 파일 태그
# 이미지 dir 경로
# annot dir 경로
# 새롭게 만들 파일 dir 이름
# 언어 ["ko", "en"]

def change_KAIST_to_ufo():

    glob_jpg = "/opt/ml/input/data/KAIST/English/**/*.JPG" # FIXME
    glob_xml = "/opt/ml/input/data/KAIST/English/**/*.xml" # FIXME
    language = "en" # FIXME
    new_file_tag = "en" # FIXME
    new_folder_name = "KAIST_EN" # FIXME
    
    print("jpg 파일 개수:",len(glob.glob(glob_jpg,recursive=True))) 
    print("xml 파일 개수:",len(glob.glob(glob_xml,recursive=True))) 

    
    img_names = []
    annot_names = []

    img_paths = glob.glob(glob_jpg,recursive=True) # FIXME
    annot_paths = glob.glob(glob_xml,recursive=True) # FIXME

    # 일단 폴더를 만듭니다
    if len(img_paths) != 0:
        os.makedirs(osp.join("/opt/ml/input/data/", f"{new_folder_name}", "images"), exist_ok=True)
        os.makedirs(osp.join("/opt/ml/input/data/", f"{new_folder_name}", "annots"), exist_ok=True)
        os.makedirs(osp.join("/opt/ml/input/data/", f"{new_folder_name}", "ufo"), exist_ok=True)
    else:
        return print("지정한 디렉토리에 jpg파일이 없습니다")
    
    
    for img_path in img_paths:
        # print(osp.basename(img_path)) # 확장자를 포함한 파일의 이름
        osp.basename(img_path).split(".")[0] # 확장자를 제외한 파일의 이름
        img_names.append(osp.basename(img_path).split(".")[0]) # 확장자를 제외한 파일의 이름
    
    for annot_path in annot_paths:
        # osp.basename(annot_path) # 확장자를 포함한 파일 이름
        osp.basename(annot_path).split(".")[0] # 확장자를 제외한 파일 이름
        annot_names.append(osp.basename(annot_path).split(".")[0]) # 확장자를 제외한 파일 이름

    

    # 이미지 기준으로 새롭게 임시 라벨을 만듭니다 임시라벨은 annots 폴더에 저장됩니다
    for idx, img_file_name, img_path in zip(range(len(img_paths)), img_names, img_paths):
        # 이미지에 annot가 존재하면
        if img_file_name in annot_names:
            # 이미지를 이름바꿔 복사
            src = img_path
            dst = osp.join("/opt/ml/input/data/", f"{new_folder_name}", "images", f"{new_file_tag}_{idx}.jpg")
            shutil.copy(src, dst)

            annot_path = "/".join(img_path.split("/")[:-1])
            # print(annot_path)
            # annot를 임시 annots 폴더로 이름바꿔 복사
            _src = osp.join(annot_path, f"{img_file_name}.xml")
            _dst = osp.join("/opt/ml/input/data/", f"{new_folder_name}", "annots", f"{new_file_tag}_{idx}.xml")
            shutil.copy(_src, _dst)
    
    annot_paths = glob.glob(f"/opt/ml/input/data/{new_folder_name}/annots/*.xml")
    new_img_paths = glob.glob(f"/opt/ml/input/data/{new_folder_name}/images/*.jpg")
    print("제대로 변환 되었는가?:", len(new_img_paths) == len(annot_paths))
    print(f"변경된 데이터 개수 {len(new_img_paths)}")
    
    ufo = dict(images=dict())
    image_names = list
    
    # annots 폴더에 있는 xml들을 하나의 json으로 만듭니다
    cnt = 0
    for annot_path in annot_paths:
        fname = osp.basename(annot_path).split(".")[0] + '.jpg'
        tree = ET.parse(annot_path)
        root = tree.getroot()
        dict_res = root.find("image").find("resolution").attrib
        dict_words = dict()

        if len(root.find("image").find("words").findall("word")) < 0:
            print('bbox가 없는 annot가 존재')
        # print(len(root.find("image").find("words").findall("word")))
        # bbox마다 돌아서 dict를 만든다
        for idx, word in enumerate(root.find("image").find("words").findall("word")):
            dict_area = word.attrib
            x1, y1 = int(dict_area["x"]), int(dict_area["y"])
            x2, y2 = int(dict_area["x"])+int(dict_area["width"]), int(dict_area["y"])
            x3, y3 = int(dict_area["x"])+int(dict_area["width"]), int(dict_area["y"])+int(dict_area["height"])
            x4, y4 = int(dict_area["x"]), int(dict_area["y"])+int(dict_area["height"])

            if int(dict_res["y"]) < y1 or y1 < 0 or \
                int(dict_res["y"]) < y2 or y2 < 0 or \
                    int(dict_res["y"]) < y3 or y3 < 0 or \
                        int(dict_res["y"]) < y4 or y4 < 0 or \
                            int(dict_res["x"]) < x1 or x1 < 0 or \
                                int(dict_res["x"]) < x2 or x2 < 0 or \
                                    int(dict_res["x"]) < x3 or x3 < 0 or \
                                        int(dict_res["x"]) < x4 or x4 < 0 :
                print("이상한 bbox 가 발견되었으므로 건너띕니다.")
                continue

            dict_words[str(idx)] = dict(points=[[x1,y1],[x2,y2],[x3,y3],[x4,y4]],
                                        transcription = "temp", 
                                        language = [f"{language}"],
                                        illegibility = False,
                                        orientation = "Horizontal" if int(dict_res["x"]) >= int(dict_res["y"]) else "Vertical",
                                        word_tags = None)
        # print(len(dict_words) is 0)
        if len(dict_words) == 0:
            cnt += 1
            print(f"{cnt}번째 제거이미지가 발견되었습니다")
            continue

        # 만들어진 하나 이미지의 bbox들을 기록
        ufo["images"][str(fname)] = dict(img_h=int(dict_res["y"]),
                                         img_w=int(dict_res["x"]),
                                         words = dict_words,
                                         tags = None)
    print("사용 가능한 이미지의 개수:", len(ufo["images"]))

    ufo_dir = osp.join("/opt/ml/input/data/", f"{new_folder_name}", "ufo")
    with open(os.path.join(ufo_dir, 'train.json'), 'w', encoding='utf-8') as f:
        json.dump(ufo, f, indent=4)

    # pprint.pprint(ufo)


In [58]:
change_KAIST_to_ufo()

jpg 파일 개수: 166
xml 파일 개수: 211
제대로 변환 되었는가?: True
변경된 데이터 개수 166
이상한 bbox 가 발견되었으므로 건너띕니다.
1번째 제거이미지가 발견되었습니다
이상한 bbox 가 발견되었으므로 건너띕니다.
사용 가능한 이미지의 개수: 165


ko  
jpg 파일 개수: 677  
xml 파일 개수: 712  
제대로 변환 되었는가?: True  
변경된 데이터 개수 663  
사용 가능한 이미지의 개수: 659  
  
en  
jpg 파일 개수: 166  
xml 파일 개수: 211  
제대로 변환 되었는가?: True  
변경된 데이터 개수 166  
사용 가능한 이미지의 개수: 165  



In [60]:
659+165

824